In [ ]:
import nd2
import numpy as np
import scipy
from numba import jit

In [ ]:
def xarr_diff(file: str = "../data/A1_s3001.nd2"):
    with nd2.ND2File(file) as f:
        xarr = f.to_xarray()
        # print(xarr.shape)
        im_diff = xarr.diff('T')
        # for index in range(0,10):
            # im_fft = np.fft.fft2(im_diff[index])
            # im_fft = scipy.fft.fft2(im_diff[index])

    return im_diff

def load_data(file: str = "../data/A1_s3001.nd2"):
    f = nd2.ND2File(file)
    return f.to_dask()

In [ ]:
%timeit xarr_diff()

In [ ]:
im_diff = xarr_diff()

In [ ]:
im_diff

In [ ]:
def dask_arr(file: str = "./data/A1_s3001.nd2"):
    with nd2.ND2File(file) as f:
        arr = f.to_dask()
        print(arr.shape)
        # im_diff = arr.diff('T')
        # # for index in range(0,10):
        #     # im_fft = np.fft.fft2(im_diff[index])
        #     # im_fft = scipy.fft.fft2(im_diff[index])

    # return im_diff

In [ ]:
def test_1():
    arr = nd2.imread("../data/A1_s3001.nd2", xarray=True, dask=True)
    d = np.diff(arr, axis=0)
    im_squared = np.zeros([512, 512])
    for frame in d:  
        im_fft = np.fft.fft2(frame)
        im_squared += np.abs(im_fft)**2
    return im_squared

def test_2():
    arr = nd2.imread("../data/A1_s3001.nd2", dask=True)
    d = np.diff(arr, axis=0)
    im_squared = np.zeros([512, 512])
    for frame in d:  
        im_fft = np.fft.fft2(frame)
        im_squared += np.abs(im_fft)**2
    return im_squared

def test_3():
    arr = nd2.imread("../data/A1_s3001.nd2", dask=True)
    arr.rechunk('auto')
    d = np.diff(arr, axis=0)
    im_fft = np.abs(np.fft.fft2(d))**2
    im_squared = im_fft.cumsum(axis=0)
    return im_squared.compute()

In [ ]:
@jit(nopython=True)
def test_numba(arr):    
    d = arr.diff()
    im_squared = np.zeros([512, 512])
    for frame in d:  
        im_fft = np.fft.fft2(frame)
        im_squared += np.abs(im_fft)**2
    return im_squared

In [ ]:
%%time
arr = nd2.imread("./data/A1_s3001.nd2")
d = np.diff(arr, axis=0)
for frame in d:  
    im_fft = np.fft.fft2(frame)

In [ ]:
@jit
def simple_numba(arr):

    num_frames = arr.shape[0]    
    for i in range(num_frames-1):
        d = arr[i+1] - arr[i]
        im_fft = np.fft.fft2(d)

In [ ]:
%%timeit
arr = nd2.imread("./data/A1_s3001.nd2")
simple_numba(arr)

In [ ]:
%%time
out_1 = test_1()

In [ ]:
%%time
out_2= test_2()

In [ ]:
%%time
out_3 = test_3()

In [ ]:
out_1

In [ ]:
out_3